# 第5章: 実践的な例題集 (Julia版)

この章では、様々な分野の実践的な線形計画問題を解きます。

In [ ]:
using JuMP
using HiGHS

## 例題1: ナップサック問題

### 問題

バックパックに入れる品物を選びます。

| アイテム | 重さ | 価値 |
|----------|------|------|
| A | 5kg | \$60 |
| B | 3kg | \$50 |
| C | 4kg | \$70 |
| D | 2kg | \$30 |
| E | 6kg | \$80 |
| F | 1kg | \$20 |
| G | 4kg | \$55 |

**バックパックの容量:** 12kg

**目標:** 価値の合計を最大化

In [ ]:
model = Model(HiGHS.Optimizer)
set_silent(model)

items = [:A, :B, :C, :D, :E, :F, :G]
weight = Dict(:A => 5, :B => 3, :C => 4, :D => 2, :E => 6, :F => 1, :G => 4)
value_item = Dict(:A => 60, :B => 50, :C => 70, :D => 30, :E => 80, :F => 20, :G => 55)
capacity = 12

# 二値変数（入れる=1, 入れない=0）
@variable(model, x[items], Bin)

# 容量制約
@constraint(model, sum(weight[i] * x[i] for i in items) <= capacity)

# 目的関数
@objective(model, Max, sum(value_item[i] * x[i] for i in items))

optimize!(model)

println("【結果】")
println("ステータス: ", termination_status(model))
println("総価値: \$", round(Int, objective_value(model)))

println("\n選択したアイテム:")
total_weight = 0
for i in items
    if value(x[i]) > 0.5
        println("  $i: 重さ", weight[i], "kg, 価値\$", value_item[i])
        total_weight += weight[i]
    end
end
println("\n使用容量: ", total_weight, "kg / ", capacity, "kg")

## 例題2: 従業員スケジューリング問題

### 問題

1週間のシフトスケジュールを作成します。

**曜日別の必要人数:** 月=5, 火=6, 水=7, 木=6, 金=8, 土=10, 日=4

**制約:**
- 各従業員は週5日連続で働く

**目標:** 必要な従業員数を最小化

In [ ]:
model = Model(HiGHS.Optimizer)
set_silent(model)

days = [:月, :火, :水, :木, :金, :土, :日]
required = Dict(:月 => 5, :火 => 6, :水 => 7, :木 => 6, :金 => 8, :土 => 10, :日 => 4)

# シフトパターン（5日連続勤務）
shifts = Dict(
    :月 => [:月, :火, :水, :木, :金],
    :火 => [:火, :水, :木, :金, :土],
    :水 => [:水, :木, :金, :土, :日],
    :木 => [:木, :金, :土, :日, :月],
    :金 => [:金, :土, :日, :月, :火],
    :土 => [:土, :日, :月, :火, :水],
    :日 => [:日, :月, :火, :水, :木]
)

# 決定変数（各シフトパターンの従業員数）
@variable(model, x[keys(shifts)] >= 0, Int)

# 各日の必要人数を満たす制約
for d in days
    @constraint(model, sum(x[s] for s in keys(shifts) if d in shifts[s]) >= required[d])
end

# 目的関数（総従業員数）
@objective(model, Min, sum(x[s] for s in keys(shifts)))

optimize!(model)

println("【結果】")
println("ステータス: ", termination_status(model))
println("必要な従業員数: ", round(Int, objective_value(model)), "人")

println("\nシフトパターン別の人数:")
for s in keys(shifts)
    v = value(x[s])
    if v > 0
        println("  $(s)曜開始: ", round(Int, v), "人 → 勤務日: ", join(shifts[s], ", "))
    end
end

println("\n曜日別の配置人数:")
for d in days
    actual = sum(value(x[s]) for s in keys(shifts) if d in shifts[s])
    println("  $d: ", round(Int, actual), "人 (必要: ", required[d], "人)")
end

## 例題3: 割当問題（タスク割当）

### 問題

4人の従業員に4つのタスクを割り当てます。
各従業員がタスクを完了するのにかかる時間（時間）:

|  | タスク1 | タスク2 | タスク3 | タスク4 |
|--|---------|---------|---------|--------|
| 従業員A | 8 | 6 | 5 | 7 |
| 従業員B | 6 | 7 | 8 | 6 |
| 従業員C | 9 | 5 | 6 | 8 |
| 従業員D | 7 | 8 | 7 | 5 |

**目標:** 総作業時間を最小化

In [ ]:
model = Model(HiGHS.Optimizer)
set_silent(model)

workers = [:A, :B, :C, :D]
tasks = [1, 2, 3, 4]

time_matrix = Dict(
    (:A, 1) => 8, (:A, 2) => 6, (:A, 3) => 5, (:A, 4) => 7,
    (:B, 1) => 6, (:B, 2) => 7, (:B, 3) => 8, (:B, 4) => 6,
    (:C, 1) => 9, (:C, 2) => 5, (:C, 3) => 6, (:C, 4) => 8,
    (:D, 1) => 7, (:D, 2) => 8, (:D, 3) => 7, (:D, 4) => 5
)

# 二値変数
@variable(model, x[workers, tasks], Bin)

# 各従業員は1つのタスクのみ
for w in workers
    @constraint(model, sum(x[w, t] for t in tasks) == 1)
end

# 各タスクは1人のみ
for t in tasks
    @constraint(model, sum(x[w, t] for w in workers) == 1)
end

# 目的関数
@objective(model, Min, sum(time_matrix[w, t] * x[w, t] for w in workers, t in tasks))

optimize!(model)

println("【結果】")
println("ステータス: ", termination_status(model))
println("総作業時間: ", round(Int, objective_value(model)), "時間")

println("\n割当結果:")
for w in workers
    for t in tasks
        if value(x[w, t]) > 0.5
            println("  従業員$w → タスク$t (", time_matrix[w, t], "時間)")
        end
    end
end

## 例題4: カッティングストック問題

### 問題

長さ10mの原材料を切断して、必要な長さの製品を作ります。

**必要な製品:**
- 3mの製品: 25本
- 4mの製品: 20本
- 5mの製品: 15本

**カットパターン:**
- P1: 3m×3本
- P2: 3m×2本 + 4m×1本
- P3: 4m×2本
- P4: 5m×2本
- P5: 3m×1本 + 4m×1本
- P6: 5m×1本 + 3m×1本
- P7: 5m×1本 + 4m×1本

**目標:** 使用する原材料の本数を最小化

In [ ]:
model = Model(HiGHS.Optimizer)
set_silent(model)

# 製品サイズと必要数
products = Dict(3 => 25, 4 => 20, 5 => 15)  # {長さ => 必要数}

# カットパターン {パターン名 => {製品長 => 本数}}
patterns = Dict(
    :P1 => Dict(3 => 3, 4 => 0, 5 => 0),
    :P2 => Dict(3 => 2, 4 => 1, 5 => 0),
    :P3 => Dict(3 => 0, 4 => 2, 5 => 0),
    :P4 => Dict(3 => 0, 4 => 0, 5 => 2),
    :P5 => Dict(3 => 1, 4 => 1, 5 => 0),
    :P6 => Dict(3 => 1, 4 => 0, 5 => 1),
    :P7 => Dict(3 => 0, 4 => 1, 5 => 1)
)

# 決定変数（各パターンを使用する回数）
@variable(model, x[keys(patterns)] >= 0, Int)

# 需要充足制約
for (length, required) in products
    @constraint(model, sum(patterns[p][length] * x[p] for p in keys(patterns)) >= required)
end

# 目的関数（使用する原材料の本数）
@objective(model, Min, sum(x[p] for p in keys(patterns)))

optimize!(model)

println("【結果】")
println("ステータス: ", termination_status(model))
println("使用する原材料: ", round(Int, objective_value(model)), "本")

println("\nカットパターンの使用回数:")
for p in keys(patterns)
    v = value(x[p])
    if v > 0
        pattern_desc = join(["$(length)m×$(count)" for (length, count) in patterns[p] if count > 0], "+")
        println("  $p: ", round(Int, v), "回 ($pattern_desc)")
    end
end

println("\n製品の生産数:")
for (length, required) in products
    produced = sum(patterns[p][length] * value(x[p]) for p in keys(patterns))
    println("  $(length)m: ", round(Int, produced), "本 (必要: ", required, "本)")
end

## 例題5: 最短経路問題

### 問題

ノードAからノードFまでの最短経路を求めます。

**ネットワーク（距離）:**
- A → B: 4,  A → C: 2
- B → C: 1,  B → D: 5
- C → D: 8,  C → E: 10
- D → E: 2,  D → F: 6
- E → F: 3

**目標:** A→Fの最短距離を求める

In [ ]:
model = Model(HiGHS.Optimizer)
set_silent(model)

# グラフの定義
edges = Dict(
    (:A, :B) => 4, (:A, :C) => 2,
    (:B, :C) => 1, (:B, :D) => 5,
    (:C, :D) => 8, (:C, :E) => 10,
    (:D, :E) => 2, (:D, :F) => 6,
    (:E, :F) => 3
)

nodes = [:A, :B, :C, :D, :E, :F]
source = :A
sink = :F

# 決定変数（各辺を使うかどうか）
@variable(model, x[keys(edges)], Bin)

# フロー保存制約
for n in nodes
    outflow = sum(x[e] for e in keys(edges) if e[1] == n; init=0)
    inflow = sum(x[e] for e in keys(edges) if e[2] == n; init=0)
    
    if n == source
        @constraint(model, outflow - inflow == 1)
    elseif n == sink
        @constraint(model, outflow - inflow == -1)
    else
        @constraint(model, outflow - inflow == 0)
    end
end

# 目的関数
@objective(model, Min, sum(edges[e] * x[e] for e in keys(edges)))

optimize!(model)

println("【結果】")
println("ステータス: ", termination_status(model))
println("最短距離: ", round(Int, objective_value(model)))

println("\n最短経路:")
for e in keys(edges)
    if value(x[e]) > 0.5
        println("  ", e[1], " → ", e[2], ": ", edges[e])
    end
end

# 経路の順序で表示
current = source
route = [current]
while current != sink
    for e in keys(edges)
        if e[1] == current && value(x[e]) > 0.5
            current = e[2]
            push!(route, current)
            break
        end
    end
end
println("\n経路: ", join(route, " → "))

## 例題6: ビンパッキング問題

### 問題

容量10の箱に、様々なサイズのアイテムを詰めます。

**アイテムのサイズ:**
1: 6,  2: 5,  3: 4,  4: 4,  5: 3,  6: 3,  7: 2,  8: 2

**目標:** 使用する箱の数を最小化

In [ ]:
model = Model(HiGHS.Optimizer)
set_silent(model)

items = 1:8
size = Dict(1 => 6, 2 => 5, 3 => 4, 4 => 4, 5 => 3, 6 => 3, 7 => 2, 8 => 2)
bin_capacity = 10

# 箱の数（最大でアイテム数と同じ）
bins = 1:length(items)

# 決定変数
@variable(model, y[bins], Bin)  # 箱を使うか
@variable(model, x[items, bins], Bin)  # アイテムiを箱bに入れるか

# 各アイテムは1つの箱に入れる
for i in items
    @constraint(model, sum(x[i, b] for b in bins) == 1)
end

# 箱の容量制約
for b in bins
    @constraint(model, sum(size[i] * x[i, b] for i in items) <= bin_capacity * y[b])
end

# 対称性を破る制約（箱を番号順に使用）
for b in 1:length(bins)-1
    @constraint(model, y[b] >= y[b+1])
end

# 目的関数
@objective(model, Min, sum(y[b] for b in bins))

optimize!(model)

println("【結果】")
println("ステータス: ", termination_status(model))
println("使用する箱の数: ", round(Int, objective_value(model)))

println("\n箱の中身:")
for b in bins
    if value(y[b]) > 0.5
        items_in_bin = [i for i in items if value(x[i, b]) > 0.5]
        sizes_in_bin = [size[i] for i in items_in_bin]
        total = sum(sizes_in_bin)
        println("  箱$b: アイテム$items_in_bin (サイズ: $sizes_in_bin, 合計: $total/$bin_capacity)")
    end
end

## 第5章のまとめ

この章で学んだ実践的な問題：

| 問題 | 特徴 |
|------|------|
| ナップサック問題 | 組合せ最適化の基本、二値変数の活用 |
| 従業員スケジューリング | シフトパターンの設計、整数変数による人数決定 |
| 割当問題 | 1対1の対応関係、行列形式のデータ |
| カッティングストック | パターン列挙法、廃棄量の最小化 |
| 最短経路問題 | グラフ上の最適化、フロー保存制約 |
| ビンパッキング | 容量制約付きの詰め込み、対称性を破る制約 |

---

## チュートリアル完了！

お疲れさまでした！このチュートリアルで学んだこと：

- **第1章:** 線形計画法の基礎概念
- **第2章:** JuMPの基本的な使い方
- **第3章:** JuMPの応用（様々な問題タイプ）
- **第4章:** 混合整数線形計画法（MILP）
- **第5章:** 実践的な例題集

### 次のステップ

- より複雑な問題への挑戦
- 他のソルバー（Gurobi, CPLEX）の試用
- 非線形計画法への拡張